In [ ]:
%load_ext autoreload
%autoreload 2

# Thinning

Decide which variable to keep and which ones to thin out in our dataset.
The objective is to save a little bit of space where possible/appropriatae.

In [ ]:
import os
import pathlib
import pygrib
import xarray as xr

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
GDPS_DIR = DATA_DIR /  'data/gdps'


In [ ]:
gdps_passes = list(pathlib.Path(GDPS_DIR).iterdir())
sample1 = list(gdps_passes[0].iterdir())[2]
sample2 = list(gdps_passes[-2].iterdir())[14]

In [ ]:
sample1

In [ ]:
grb = pygrib.open(str(sample1))

In [ ]:
m = next(iter(grb))

In [ ]:
sorted(m.keys())

In [ ]:
m.paramId

In [ ]:
grb.seek(340)
m = next(grb)
sorted(m.keys())

In [ ]:
grb.seek(0)
for i, m in enumerate(grb):
    print(m)

In [ ]:
grb.seek(0)
for i, m in enumerate(grb):
    if m.levelType == 'pl' and m.level not in [1000, 925, 850, 700, 500, 250]:
        print(m)

In [ ]:
grb.seek(0)
for i, m in enumerate(grb):
    print(i, m.name, m.localTablesVersion, m.parameterCategory, m.parameterName, m.parameter, m.paramId, m.unitsECMF, 'level', m.level)

## With ECCODES

In [ ]:
import eccodes

In [ ]:
try:
    infile = open(sample2, 'rb')
    
    keep_going = True
    while keep_going:
        msg_begin = infile.tell()
        msg_id = eccodes.codes_grib_new_from_file(infile)
        if msg_id:
            typeOfLevel = eccodes.codes_get(msg_id, 'typeOfLevel')
            level = eccodes.codes_get(msg_id, 'level')
            print(level, typeOfLevel)
        else:
            keep_going = False
finally:
    infile.close()

In [ ]:
from smc01.grib import GribFile

In [ ]:
sample2

In [ ]:
with GribFile(sample2, 'rb') as f:
    for msg in f:
        print(msg)

In [ ]:
with open('outfile.grib2', 'wb') as outfile:
    with GribFile(sample2) as f:
        for msg in f:
            if msg['typeOfLevel'] == 'isobaricInhPa' and msg['level'] not in [1000, 925, 850, 700, 500]:
                print(msg)
            else:
                eccodes.codes_write(msg.msg_id, outfile)

In [ ]:
with GribFile('outfile.grib2', 'rb') as f:
    msgs = list(f)

In [ ]:
outfile_path = pathlib.Path('outfile.grib2')

In [ ]:
if outfile_path.is_file():
    outfile_path.unlink()

In [ ]:
msgs[0]['level']
